# get_frequently_mutated Final 

In [3]:
import cptac
#en = cptac.Endometrial()
#cptac.sync('endometrial', version='2.1')
co = cptac.Colon()
#cptac.sync(dataset='ovarian', version='0.0')
#cptac.sync(dataset='colon', version='0.0')
#ov = cptac.Ovarian()

import pandas as pd
import numpy as np

In [14]:
def get_frequently_mutated(cancer_object, cutoff = 0.1):  
    """
    take cancer object and find the frequently 
    mutated genes in the total tumors compared to the cutoff.

    Parameters:
    cancer_object (object): cancer class object from cptac module 
    cutoff (float): used as comparison to determine the 
                    status of gene mutation frequency

    Returns:
    freq_mutated_df (pd.DataFrame): DataFrame of frequently 
        mutated genes passing the cutoff. Columns contain the 
        fractions of total unique mutations,missence type 
        mutations, and truncation type mutations per gene.
    
    The Missence_Mut column includes: 
        In_Frame_Del, In_Frame_Ins, Missense_Mutation
   
   The Truncation_Mut column includes: 
        Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, 
        Nonsense_Mutation, Nonstop_Mutation
        
    These columns count multiple mutations of one gene in the 
    same sample, so fractions in the last two columns may 
    exceed the Unique_Samples_Mut column which only counts if 
    the gene was mutated once per sample."""    
    
    # Step 1 - Get data frames and total tumor count
    somatic_mutations = cancer_object.get_mutations() 
    maps = cancer_object._get_sample_status_map()
    t = somatic_mutations.join(maps, how= 'left')

    omics_and_mutations = cancer_object.append_mutations_to_omics(
        mutation_genes = 'TP53', omics_df_name = 'proteomics', omics_genes = 'TP53')
    tumors = omics_and_mutations.loc[omics_and_mutations['Sample_Status'] == 'Tumor']
    total_tumor_patients = len(tumors)
    print(total_tumor_patients)
    
    # Step 2 - Find frequently mutated genes and their fraction of unique mutated samples.

    # Drop silent mutations for Ovarian dataset
    if cancer_object.get_cancer_type() == 'ovarian':
        origin_df = origin_df.loc[origin_df['Mutation'] != 'Silent']
        
    #group by gene and count unique samples
    origin_df = somatic_mutations.reset_index() #move 'Sample_ID' in order to count sample labels
    count_mutations = origin_df.groupby(['Gene']).nunique()
    #format
    count_mutations = count_mutations.rename(columns={"Sample_ID": "Unique_Samples_Mut"})
    count_mutations = count_mutations.drop(['Gene', 'Mutation','Location'], axis = 1)
    #filter using the cutoff and create fraction
    fraction_mutated = count_mutations.apply(lambda x: x / total_tumor_patients)
    fraction_greater_than_cutoff = fraction_mutated.where(lambda x: x > cutoff) #na used when false
    filtered_gene_df = fraction_greater_than_cutoff.dropna()
    
    # Step 3 - Create Missence and Trucation data frame
    if cancer_object.get_cancer_type() == 'colon':
        missence_truncation_groups = {'frameshift substitution': 'T', 
            'frameshift deletion': 'T', 'frameshift insertion': 'T', 
            'stopgain': 'T', 'stoploss': 'T', 'nonsynonymous SNV': 'M',
            'nonframeshift insertion': 'M','nonframeshift deletion': 'M', 
            'nonframeshift substitution': 'M'}
    else: 
        missence_truncation_groups = {'In_Frame_Del': 'M', 'In_Frame_Ins': 'M',
            'Missense_Mutation': 'M', 'Frame_Shift_Del': 'T','Nonsense_Mutation': 'T', 
            'Splice_Site': 'T', 'Frame_Shift_Ins': 'T','Nonstop_Mutation':'T'}
    mutations_replaced_M_T = origin_df.replace(missence_truncation_groups)
    
    # group mutation categories
    miss = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'M']
    trunc = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'T']
    # group by gene and count unique samples for both categories
    count_miss = miss.groupby(['Gene']).nunique()
    count_trunc = trunc.groupby(['Gene']).nunique()
    #format
    missence_df = count_miss.rename(columns={"Sample_ID": "Missence_Mut"})
    missence_df = missence_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)
    truncation_df = count_trunc.rename(columns={"Sample_ID": "Truncation_Mut"})
    truncation_df = truncation_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)

    # Step 4 - Combine the dataframes 
    #join miss and trunc, change nan to 0, then divide by total tumors
    join_mutations = missence_df.join(truncation_df).fillna(0)
    missence_and_truncation_df = join_mutations.apply(lambda x: x / total_tumor_patients)
    #Join data frames, keeping only the genes that passed the cutoff 
    freq_mutated_df = filtered_gene_df.join(missence_and_truncation_df).reset_index()
    freq_mutated_df.name = 'frequently_mutated'
    
    return t

In [28]:
somatic_mutations = co.get_mutations() 
maps = co._get_sample_status_map()
t = somatic_mutations.join(maps, how= 'left')
g = t.loc[t['Gene'] == 'TP53']
wt = g.loc[g['Sample_Status'] != 'Tumor']
len(g.index.unique())

56

In [15]:
df = get_frequently_mutated(co, .15)

97


In [20]:
df
df.loc[df['Gene'] == 'TP53']

,Gene,Mutation,Location,Sample_Status
Sample_ID,,,,
S001,TP53,nonsynonymous SNV,G113S,Tumor
S001,TP53,nonsynonymous SNV,G113S,Tumor
S001,TP53,nonsynonymous SNV,G113S,Tumor
S001,TP53,nonsynonymous SNV,G206S,Tumor
S001,TP53,nonsynonymous SNV,G206S,Tumor
S001,TP53,nonsynonymous SNV,G206S,Tumor
S001,TP53,nonsynonymous SNV,G206S,Tumor
S001,TP53,nonsynonymous SNV,G206S,Tumor
S001,TP53,nonsynonymous SNV,G245S,Tumor


In [3]:
co = cptac.Colon()

In [11]:
colon_freq_mutated_df = get_frequently_mutated(co, .15)

           TP53_proteomics                                      TP53_Mutation  \
Sample_ID                                                                       
S002               -0.4890  [frameshift insertion, frameshift insertion, f...   
S003               -0.7960                                   [Wildtype_Tumor]   
S004                   NaN                                   [Wildtype_Tumor]   
S005               -1.3900  [nonsynonymous SNV, nonsynonymous SNV, nonsyno...   
S006                   NaN  [nonsynonymous SNV, nonsynonymous SNV, nonsyno...   
S007                   NaN                                   [Wildtype_Tumor]   
S008                   NaN  [nonsynonymous SNV, nonsynonymous SNV, nonsyno...   
S009               -0.2630                                   [Wildtype_Tumor]   
S010                   NaN  [nonsynonymous SNV, nonsynonymous SNV, nonsyno...   
S011                   NaN                                   [Wildtype_Tumor]   
S014                   NaN  

In [12]:
colon_freq_mutated_df

,Gene,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
0,ABCA13,0.195876,0.164948,0.103093
1,ABCA2,0.175258,0.164948,0.030928
2,ACVR2A,0.257732,0.020619,0.257732
3,ADGRV1,0.195876,0.134021,0.092784
4,AHNAK2,0.216495,0.195876,0.030928
5,ANK1,0.154639,0.123711,0.030928
6,ANK2,0.154639,0.134021,0.041237
7,ANKRD11,0.154639,0.123711,0.041237
8,APC,0.845361,0.113402,0.824742
9,APOB,0.195876,0.175258,0.061856


ValueError: too many values to unpack (expected 1)

In [6]:
#test results
m = co.get_mutations()
gene = 'CASP5'
g = m.loc[m['Gene'] == gene]
print(g)
print(len(g.index.unique()))

r = g.groupby('Mutation')

r.groups


            Gene                 Mutation    Location
Sample_ID                                            
S006       CASP5  frameshift substitution     K9Nfs*2
S006       CASP5  frameshift substitution    K80Nfs*2
S006       CASP5  frameshift substitution    K67Nfs*2
S009       CASP5  frameshift substitution    K67Nfs*2
S009       CASP5  frameshift substitution   T68Qfs*25
S009       CASP5  frameshift substitution   T10Qfs*25
S009       CASP5  frameshift substitution   T81Qfs*25
S009       CASP5  frameshift substitution    K80Nfs*2
S009       CASP5  frameshift substitution     K9Nfs*2
S014       CASP5  frameshift substitution   T10Qfs*25
S014       CASP5  frameshift substitution   T68Qfs*25
S014       CASP5  frameshift substitution   T81Qfs*25
S018       CASP5  frameshift substitution    K67Nfs*2
S018       CASP5  frameshift substitution   T68Qfs*25
S018       CASP5  frameshift substitution   T81Qfs*25
S018       CASP5  frameshift substitution   T10Qfs*25
S018       CASP5  frameshift

{'frameshift deletion': Index(['S055', 'S055', 'S055', 'S055', 'S074', 'S074', 'S074', 'S080', 'S080',
        'S080', 'S081', 'S081'],
       dtype='object', name='Sample_ID'),
 'frameshift insertion': Index(['S031', 'S031', 'S031', 'S031'], dtype='object', name='Sample_ID'),
 'frameshift substitution': Index(['S006', 'S006', 'S006', 'S009', 'S009', 'S009', 'S009', 'S009', 'S009',
        'S014', 'S014', 'S014', 'S018', 'S018', 'S018', 'S018', 'S018', 'S018',
        'S023', 'S023', 'S023', 'S023', 'S023', 'S023', 'S023', 'S023', 'S028',
        'S028', 'S028', 'S028', 'S030', 'S030', 'S030', 'S033', 'S033', 'S033',
        'S033', 'S035', 'S035', 'S035', 'S035', 'S035', 'S052', 'S052', 'S052',
        'S052', 'S052', 'S052', 'S052', 'S052', 'S052', 'S052', 'S055', 'S055',
        'S055', 'S055', 'S064', 'S064', 'S064', 'S065', 'S065', 'S065', 'S065',
        'S065', 'S065', 'S065', 'S065', 'S074', 'S074', 'S074', 'S080', 'S080',
        'S080', 'S101', 'S101', 'S101', 'S108', 'S108',

In [19]:
en_freq_mutated_df = get_frequently_mutated(en,.1)

95


In [ ]:
ovarian_freq_mutated_df